## 1. 视频转换图片

In [3]:
VIDEO_PATH = 'D:/0LT/chitanda/vedio/05.mkv' # 视频地址
EXTRACT_FOLDER = 'D:/0LT/chitanda/vedio_pic/' # 存放帧图片的位置
EXTRACT_FREQUENCY = 100 # 帧提取频率

def extract_frames(video_path, dst_folder, index):
    # 主操作
    import cv2
    video = cv2.VideoCapture()
    if not video.open(video_path):
        print("can not open the video")
        exit(1)
    count = 1
    while True:
        _, frame = video.read()
        if frame is None:
            break
        if count % EXTRACT_FREQUENCY == 0:
            save_path = "{}/{:>04d}.jpg".format(dst_folder, index)
            cv2.imwrite(save_path, frame)
            index += 1
        count += 1
    video.release()
    # 打印出所提取帧的总数
    print("Totally save {:d} pics".format(index-1))
    
if __name__ == '__main__':
    extract_frames(VIDEO_PATH, EXTRACT_FOLDER, 1497)


Totally save 1909 pics


## 2. 手动将不需要的图片删除，修改整理好后的图片名称（有序排列）

rename()不能修改为已经有的名字，会报错
当文件夹内有新旧（打过标签和未打标签）图片时，排序并非看起来的数字排序，所以可能未达标签的顺序可能比较靠前，使得重新命名的时候会命名为
已经存在的名字，从而报错。
所以最好将打过标签的图片存放在别的文件夹内，等这边的修改完再考进来（此时修改要记住命名的起点不是1）。

In [15]:
import os  
      
class BatchRename():  
        #批量重命名文件夹中的图片文件 
        def __init__(self):  
            self.path = 'D:/0LT/chitanda/VOC2007/JPEGImages/'  #图片文件夹路径
      
        def rename(self):  
            filelist = os.listdir(self.path)  
            total_num = len(filelist)  
            i = 816  #从1开始排序
            for item in filelist:  
                if item.endswith('.jpg'):  
                    n = 3 - len(str(i))  
                    src = os.path.join(os.path.abspath(self.path), item)  
                    dst = "{}{:03d}.jpg".format(self.path, i) 
                    try:  
                        os.rename(src, dst)  
                        i = i + 1 
                    except:  
                        print(src,dst)
                        break  
            print('total %d to rename & converted %d jpgs' % (total_num, i))  
      
if __name__ == '__main__':  
        demo = BatchRename()  
        demo.rename()  

total 42 to rename & converted 858 jpgs


## 3. labelimg给图片打上标签 

## 4. 划分数据集

In [16]:
import os
import random

xmlfilepath = r'D:/0LT/chitanda/VOC2007/Annotations/'
saveBasePath = r'D:/0LT/chitanda/VOC2007/ImageSets/'

trainval_percent = 0.8
train_percent = 0.8
total_xml = os.listdir(xmlfilepath)
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)

print("train and val size", tv)
print("train size", tr)
ftrainval = open(os.path.join(saveBasePath, 'Main/trainval.txt'), 'w')
ftest = open(os.path.join(saveBasePath, 'Main/test.txt'), 'w')
ftrain = open(os.path.join(saveBasePath, 'Main/train.txt'), 'w')
fval = open(os.path.join(saveBasePath, 'Main/val.txt'), 'w')

for i in list:
    name = total_xml[i][:-3] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

train and val size 685
train size 548


## 4. 将图像数据修改tf格式

### 4.1 按照自己的数据集类型修改文件：SSD-Tensorflow/datasets/pascalvoc_common.py，示例如下： 

VOC_LABELS = {
    'none': (0, 'Background'),
    'Chitanda': (1, 'People'),
    'Oreki': (2, 'People'),
    'Fukube': (3, 'People'),
    'Ibara': (4, 'People'),
}

### 4.2 新建tfrecords_文件，保存tf格式的数据

### 4.3 在pycharm里面运行tf_convert_data.py，输入如下参数.第一行和第三行不用更改，二四两行要改成绝对地址。

--dataset_name=pascalvoc
--dataset_dir=D:/0LT/chitanda/VOC2007/
--output_name=voc_2007_train
--output_dir=D:/0LT/chitanda/VOC2007/tfrecords_/

## 5.训练模型 

### 5.1 统计数据集的一些信息

In [18]:
import re
import os
import xml.etree.ElementTree as ET

class1 = 'Chitanda'
class2 = 'Oreki'
class3 = 'Fukube'
class4 = 'Ibara'
annotation_folder = 'D:/0LT/chitanda/VOC2007/Annotations/'  # 改为自己标签文件夹的路径
list = os.listdir(annotation_folder)
def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.xml':
                L.append(os.path.join(root, file))
    return L
total_number1 = 0
total_number2 = 0
total_number3 = 0
total_number4 = 0
total = 0
total_pic = 0
pic_num1 = 0
pic_num2 = 0
pic_num3 = 0
pic_num4 = 0
flag1 = 0
flag2 = 0
flag3 = 0
flag4 = 0
xml_dirs = file_name(annotation_folder)

for i in range(0, len(xml_dirs)):
    #print(xml_dirs[i])
    annotation_file = open(xml_dirs[i]).read()
    root = ET.fromstring(annotation_file)
    total_pic = total_pic + 1
    for obj in root.findall('object'):
        label = obj.find('name').text
        if label == class1:
            total_number1 = total_number1 + 1
            flag1 = 1
            total = total + 1
        # print("bounding box number:", total_number1)
        if label == class2:
            total_number2 = total_number2 + 1
            flag2 = 1
            total = total + 1
        if label == class3:
            total_number3 = total_number3 + 1
            flag3 = 1
            total = total + 1
        if label == class4:
            total_number4 = total_number4 + 1
            flag4 = 1
            total = total + 1
    if flag1 == 1:
        pic_num1 = pic_num1 + 1
        # print("pic number:", pic_num1)
        flag1 = 0
    if flag2 == 1:
        pic_num2 = pic_num2 + 1
        flag2 = 0
    if flag3 == 1:
        pic_num3 = pic_num3 + 1
        flag3 = 0
    if flag4 == 1:
        pic_num4 = pic_num4 + 1
        flag4 = 0
print(class1, pic_num1, total_number1)
print(class2, pic_num2, total_number2)
print(class3, pic_num3, total_number3)
print(class4, pic_num4, total_number4)
print("total", total_pic, total)

Chitanda 499 499
Oreki 558 558
Fukube 289 289
Ibara 228 228
total 857 1574


### 5.2 按照上面的结果修改pascalvoc_2007.py

TRAIN_STATISTICS = {
    'none': (0, 0),
    'Chitanda': (289, 289),
    'Oreki': (335, 335),
    'Fukube': (191, 191),
    'Ibara': (105, 105),
    'total': (460, 920),
}
TEST_STATISTICS = {
    'none': (0, 0),
    'Chitanda': (1, 1),
    'Oreki': (1, 1),
    'Fukube': (1, 1),
    'Ibara': (1, 1),
    'total': (4, 4),
}
SPLITS_TO_SIZES = {
    'train': 294,
    'test': 92,
}
SPLITS_TO_STATISTICS = {
    'train': TRAIN_STATISTICS,
    'test': TEST_STATISTICS,
}
NUM_CLASSES = 4

### 5.3 修改ssd_vgg_300.py     96，97行

 num_classes=5,         #根据自己的数据修改为类别+1 
 no_annotation_label=5, #根据自己的数据修改为类别+1

### 5.4 修改eval_ssd_network.py    66行

tf.app.flags.DEFINE_integer('num_classes', 5, 'Number of classes to use in the dataset.')   #类别+1

### 5.5 修改 train_ssd_network.py     35，142，50-74，158，162 行

## 6.训练模型

同上，在pycharm里面运行，train_ssd_network.py 输入如下参数，修改地址即可，其它酌情修改。 

### 6.1从零开始

--train_dir=D:/0LT/SSD_test/checkpoints/
--dataset_dir=D:/0LT/chitanda/VOC2007/tfrecords_/
--dataset_name=pascalvoc_2007
--dataset_split_name=train
--model_name=ssd_300_vgg
--save_summaries_secs=600
--save_interval_secs=600
--optimizer=adam
--learning_rate=0.001
--learning_rate_decay_factor=0.94
--batch_size=4
--gpu_memory_fraction=0.9

#备注 batch_size不要过大，之前为32，会报错GPU内存不够，现改为8

### 6.2加载现有模型

--train_dir=D:/0LT/SSD_test/checkpoints/           
--dataset_dir=D:/0LT/chitanda/VOC2007/tfrecords_/      
--dataset_name=pascalvoc_2007        
--dataset_split_name=train   
--model_name=ssd_300_vgg              
--checkpoint_path=D:/0LT/SSD_test/checkpoints/vgg_16.ckpt
--checkpoint_model_scope=vgg_16 
--checkpoint_exclude_scopes=ssd_300_vgg/conv6,ssd_300_vgg/conv7,ssd_300_vgg/block8,ssd_300_vgg/block9,ssd_300_vgg/block10,ssd_300_vgg/block11,ssd_300_vgg/block4_box,ssd_300_vgg/block7_box,ssd_300_vgg/block8_box,ssd_300_vgg/block9_box,ssd_300_vgg/block10_box,ssd_300_vgg/block11_box  
--trainable_scopes=ssd_300_vgg/conv6,ssd_300_vgg/conv7,ssd_300_vgg/block8,ssd_300_vgg/block9,ssd_300_vgg/block10,ssd_300_vgg/block11,ssd_300_vgg/block4_box,ssd_300_vgg/block7_box,ssd_300_vgg/block8_box,ssd_300_vgg/block9_box,ssd_300_vgg/block10_box,ssd_300_vgg/block11_box  
--save_summaries_secs=600 
--save_interval_secs=600  
--weight_decay=0.0005  
--optimizer=adam        
--learning_rate=0.0001
--learning_rate_decay_factor=0.94
--batch_size=4                   
--gpu_memory_fraction=0.9    